In [ ]:
# Load dependencies
import pandas as pd
import numpy as np
import gdal
from scipy.stats import gmean
import sys
sys.path.insert(0,'../../statistics_helper/')
from fraction_helper import *
from CI_helper import *
from excel_utils import *
pd.options.display.float_format = '{:,.1e}'.format

# Estimating the biomass of marine fungi

## Epipelagic fungi
### qPCR-based method
### Direct count method

## Deep-sea fungi


## Particle attached fungi
##